In [1]:
from particle import Particle
from algorithms import BarnesHut, PairWise, FMM
from forces import Inverse
import numpy as np
from universe import Universe
import matplotlib.pyplot as plt
np.random.seed(0)

## Errors in Acceleration

### Barnes-Hut Acceleration Error

In [ ]:
Ns = [10, 100, 1000]
ITERS = 40
THETAS = np.linspace(0, 1, 11)
G = 1

force = Inverse(G)
BH_algorithm = BarnesHut(force)
PW_algorithm = PairWise(force)

errors = {N: [] for N in Ns}
errors_std = {N: [] for N in Ns}

for N in Ns:
    print(N)
    for theta in THETAS:
        BH_algorithm.theta = theta

        temp_errors = []
        for _ in range(ITERS):
            particles = [Particle(charge=1/np.sqrt(N)) for _ in range(N)]

            BH_accelerations = BH_algorithm.calculate_accelerations(particles)
            PW_accelerations = PW_algorithm.calculate_accelerations(particles)

            diff = BH_accelerations - PW_accelerations
            error = np.mean(np.abs(diff))
            temp_errors.append(error)

        errors[N].append(np.mean(temp_errors))
        errors_std[N].append(np.std(temp_errors))       

### FMM Acceleration Error

In [ ]:
N = 100
MAX_DEPTH = 3
G = 1
PRECISIONS = np.arange(1, 31, 2)
ITERS = 10

force = Inverse(G)

PW_algorithm = PairWise(force)
particles = [[Particle(charge=1/np.sqrt(N)) for _ in range(N)] 
             for _ in range(ITERS)]

In [ ]:
errors = []
errors_std = []

for precision in PRECISIONS:
    print(precision)
    temp_errors = []
    for i in range(ITERS):
        FMM_algorithm = FMM(MAX_DEPTH, precision, G)

        FMM_accelerations = FMM_algorithm.calculate_accelerations(particles[i])
        PW_accelerations = PW_algorithm.calculate_accelerations(particles[i])

        diff = FMM_accelerations - PW_accelerations
        error = np.mean(np.abs(diff))
        temp_errors.append(error)
    errors.append(np.mean(temp_errors))
    errors_std.append(np.std(temp_errors))

## Errors in Momentum

In [2]:
N = 100
G = 1
DT = 0.01
MAX_DEPTH = int(np.log(N) / np.log(4))
PRECISION = 4
THETA=0.5
TIME_STEPS = 100

force = Inverse(G)
init_particles = [Particle(charge=1/np.sqrt(N)) for _ in range(N)]

### Pair-Wise Momentum Error

In [ ]:
particles = [particle.copy() for particle in init_particles]
PW_algorithm = PairWise(force)
PW_universe = Universe(particles, PW_algorithm, DT)

PW_momentums = []
times = []
for _ in range(TIME_STEPS):
    PW_momentums.append(abs(PW_universe.calculate_momentum()))
    times.append(PW_universe.T)

    PW_universe.update()

### Barnes-Hut Momentum Error

In [ ]:
particles = [particle.copy() for particle in init_particles]
BH_algorithm = BarnesHut(force, theta=THETA)
BH_universe = Universe(particles, BH_algorithm, DT)

BH_momentums = []
for _ in range(TIME_STEPS):
    BH_momentums.append(abs(BH_universe.calculate_momentum()))

    BH_universe.update()

### FMM Momentum Error

In [ ]:
particles = [particle.copy() for particle in init_particles]
FMM_algorithm = FMM(MAX_DEPTH, PRECISION, G)
FMM_universe = Universe(particles, FMM_algorithm, DT)

FMM_momentums = []
for _ in range(TIME_STEPS):
    FMM_momentums.append(abs(FMM_universe.calculate_momentum()))

    FMM_universe.update()

## Errors in Energy

In [3]:
np.random.seed(0)
N = 100
G = -1
DT = 0.005
MAX_DEPTH = int(np.log(N) / np.log(4))
PRECISION = 4
THETA=0.5
TIME_STEPS = 1000

force = Inverse(G)
init_particles = [Particle(charge=1/np.sqrt(N)) for _ in range(N)]

### Pair-Wise Energy Error

In [4]:
particles = [particle.copy() for particle in init_particles]
PW_algorithm = PairWise(force)
PW_universe = Universe(particles, PW_algorithm, DT)

PW_kinetic = []
PW_potential = []
times = []
for _ in range(TIME_STEPS):
    PW_kinetic.append(PW_universe.calculate_kinetic_energy())
    PW_potential.append(PW_universe.calculate_potential())
    times.append(PW_universe.T)

    PW_universe.update()

### Barnes-Hut Energy Error

In [6]:
particles = [particle.copy() for particle in init_particles]
BH_algorithm = BarnesHut(force, theta=THETA)
BH_universe = Universe(particles, BH_algorithm, DT)

BH_kinetic = []
BH_potential = []
times = []
for _ in range(TIME_STEPS):
    BH_kinetic.append(BH_universe.calculate_kinetic_energy())
    BH_potential.append(BH_universe.calculate_potential())
    times.append(BH_universe.T)

    BH_universe.update()

### FMM Energy Error

In [8]:
particles = [particle.copy() for particle in init_particles]
FMM_algorithm = FMM(MAX_DEPTH, PRECISION, G)
FMM_universe = Universe(particles, FMM_algorithm, DT)

FMM_kinetic = []
FMM_potential = []
times = []
for _ in range(TIME_STEPS):
    FMM_kinetic.append(FMM_universe.calculate_kinetic_energy())
    FMM_potential.append(FMM_universe.calculate_potential())
    times.append(FMM_universe.T)

    FMM_universe.update()